In [1]:
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import TfidfModel

import spacy
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim

In [2]:
english_stopwords = stopwords.words("english")
data = pd.read_csv(r"C:\Users\alkrd\Desktop\graduation_project\the_project\preprocssed_data\cleaned_data_USA.csv")
dataFrame = data[['newsSnippet']]

In [3]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    print("lemmatizing...")
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    text_out = []
    for text in texts:
        doc = nlp(text)
        new_text = [token.lemma_ for token in doc if token.pos_ in allowed_postags]
        final = " ".join(new_text)
        text_out.append(final)
    return text_out

lemmatized_texts = lemmatization(dataFrame['newsSnippet'].astype(str).tolist())
print("lemmitized sample: ", lemmatized_texts[100])

lemmatizing...
lemmitized sample:  championship game feature accord most ranking good team result flash brilliance sloppy play defeat night


In [4]:
def gen_words(texts):
    print("removing stop words...")
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)
print("after removing stop words: ",data_words[100])

removing stop words...
after removing stop words:  ['championship', 'game', 'feature', 'accord', 'most', 'ranking', 'good', 'team', 'result', 'flash', 'brilliance', 'sloppy', 'play', 'defeat', 'night']


In [5]:
bigrams_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigrams_phrases[data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return(bigram[doc] for doc in texts)

def make_tigram(texts):
    return (trigram[bigram[doc]] for doc in texts)

data_bigrams = list(make_bigrams(data_words))
data_bigrams_trigrams = list(make_tigram(data_bigrams))

print(data_bigrams_trigrams)



[['logo', 'stock', 'buy', 'rating', 'restate', 'analyst', 'research', 'note', 'issue', 'currently', 'target', 'price', 'apparel', 'retailer', 'stock'], ['dick', 'expand', 'digital', 'tool', 'acquisition', 'company', 'mobile', 'scorekeeping', 'app', 'same', 'name', 'sporting', 'good', 'company', 'location', 'seem'], ['service', 'directv', 'now', 'include', 'many', 'channel', 'include', 'socalle', 'cordcutter', 'frequently', 'miss', 'ditch', 'cable', 'live', 'television', 'stream', 'mobile', 'device', 'tablet', 'computer'], ['see', 'playoff', 'hope', 'take', 'major', 'blow', 'night', 'many', 'way', 'only', 'blame', 'overcome', 'shoddy', 'defense', 'give', 'average', 'point', 'game'], ['announce', 'release', 'pc', 'develop', 'board', 'game', 'digital', 'version', 'realworld', 'board', 'game', 'create', 'player', 'become', 'train'], ['about', 'dozen', 'hour', 'slain', 'countless', 'monster', 'travel', 'hundred', 'mile', 'bond', 'tightknit', 'group', 'friend', 'strange', 'feeling', 'swell',

In [6]:

id2word = corpora.Dictionary(data_bigrams_trigrams)
texts = data_bigrams_trigrams
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[0])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.01
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]  
    corpus[i] = new_bow

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1)]


In [7]:
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=5,
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=20,
    alpha="auto"
)

In [8]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.339210 -0.035701       1        1  28.742748
4      0.101128  0.291957       2        1  26.796168
0     -0.254099 -0.168431       3        1  17.229281
3     -0.233220  0.172770       4        1  14.966537
2      0.046981 -0.260596       5        1  12.265266, topic_info=         Term        Freq       Total Category  logprob  loglift
200       say  393.000000  393.000000  Default  30.0000  30.0000
312      make  182.000000  182.000000  Default  29.0000  29.0000
248       new  179.000000  179.000000  Default  28.0000  28.0000
502     first  254.000000  254.000000  Default  27.0000  27.0000
366      look  152.000000  152.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
1253  message   27.498485   28.226207   Topic5  -5.3603   2.0723
2080     mark   25.728961   26.441980   Topic5  -5.4268   2.0711
1011     race   25.555656   26.276896   Topic5  -5.4336   2.0706
2274   public   23.197107   23.916342   Topic5  -5.5304   2.0679
567       fan   34.034503  133.782783   Topic5  -5.1471   0.7296

[187 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
75        5  0.984538     about
608       2  0.990303    accord
618       4  0.984907     actor
703       4  0.993152   actress
278       2  0.998507      also
...     ...       ...       ...
1067      2  0.986606      work
410       3  0.989124     world
888       4  0.966790     write
228       1  0.995654      year
524       3  0.985998  year_ago

[162 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 1, 4, 3])